In [1]:
import gzip
import sqlite3 as sq

# Populate the Database

- Read the '.gz' archive file [downloaded here](http://storage.googleapis.com/books/ngrams/books/datasetsv2.html)
- Fill a table with the informations: ngram, year, match_count, volume_count, bulk, cat

`bulk` est le ngram sans la categorie grammatical, et en minuscule.
`cat` est la categorie grammatical.

In [2]:
letter = 'r'

databasefilename = 'data/database-%s.sqlite' % letter

In [3]:
%rm $databasefilename

rm: cannot remove 'data/database-r.sqlite': No such file or directory


In [4]:
dbconnection = sq.connect('data/database-%s.sqlite'%letter)
cursor = dbconnection.cursor()

In [5]:
cursor.execute( '''DROP TABLE IF EXISTS ngram''' )

In [6]:
createTable = """ CREATE TABLE ngram (
                    id INTEGER PRIMARY KEY,
                    ngram TEXT,
                    year INTEGER,
                    match_count INTEGER,
                    volume_count INTEGER,
                    bulk TEXT,
                    cat TEXT)
                """

In [7]:
cursor.execute( createTable )

In [8]:
def parseLine( line ):
    ''' from the google doc:
        ngram TAB year TAB match_count TAB volume_count NEWLINE
    '''
    L = line.split('\t')
    
    ngram = L[0]
    ngram_splited = ngram.split('_')
    if len(ngram_splited)==2:
        bulk = ngram_splited[0].lower()
        cat = ngram_splited[1]
    else:
        bulk = ngram.lower()
        cat = None
    
    dico = {'ngram':ngram,
            'year':int(L[1]),
            'match_count':int(L[2]),
            'volume_count':int(L[3]),
            'bulk':bulk,
            'cat':cat}
    
    return dico

In [9]:
def insertAline( cursor, entry ):
    insertquery = '''INSERT INTO ngram (
                     ngram, year, match_count, volume_count, bulk, cat )
                     VALUES(?, ?, ?, ?, ?, ?) '''

    cursor.execute(insertquery, entry)
    
def insertManylines( cursor, entries ):
    insertquery = '''INSERT INTO ngram (
                     ngram, year, match_count, volume_count, bulk, cat )
                     VALUES(?, ?, ?, ?, ?, ?) '''

    cursor.executemany(insertquery, entries)

In [10]:
filepath = 'data/'+'googlebooks-fre-all-1gram-20120701-%s.gz' % letter

In [11]:
# Command to get the number of lines:
Ntot = !zcat $filepath | wc -l
Ntot = int(Ntot[0])
print(Ntot)
print('%e'%Ntot)

35780570
3.578057e+07


In [12]:
Nmax = Ntot # int( Ntot/3 )
bufferSize = 50000
buffer = []

with gzip.open(filepath, 'rt') as textfile:  
    
    for k in range( Nmax ):
        line = textfile.readline()
        d = parseLine(line)
        
        if d['year'] < 1800:
            continue
            
        entry = [d['ngram'], d['year'], d['match_count'], d['volume_count'], 
                 d['bulk'], d['cat']]
        buffer.append( entry )
        
        if k % bufferSize == 0:
            insertManylines( cursor, buffer )
            buffer = []
            percent = k/Ntot*100
            print( '\r %.3f %%'%percent, end='' )
        
dbconnection.commit()
print('done')

 99.915 %done


In [13]:
createIndex = "CREATE INDEX indexNgram ON ngram (ngram);"
cursor.execute(createIndex)

In [14]:
createIndex = "CREATE INDEX indexBulk ON ngram (bulk);"
cursor.execute(createIndex)

In [15]:
cursor.close()